In [2]:
## Week_2 Solutions ##

In [3]:
## Closed Form Solution for Linear Regression##

In [4]:
def closed_form(X, Y, lambda_factor):
    """
    Computes the closed form solution of linear regression with L2 regularization

    Args:
        X - (n, d + 1) NumPy array (n datapoints each with d features plus the bias feature in the first dimension)
        Y - (n, ) NumPy array containing the labels (a number from 0-9) for each
            data point
        lambda_factor - the regularization constant (scalar)
    Returns:
        theta - (d + 1, ) NumPy array containing the weights of linear regression. Note that theta[0]
        represents the y-axis intercept of the model and therefore X[0] = 1
    """
    # YOUR CODE HERE
    import numpy as np
    theta=np.dot(np.linalg.inv(np.dot(np.transpose(X),X)+lambda_factor*np.identity(X.shape[1])),(np.dot(np.transpose(X),Y)))
    return theta
    raise NotImplementedError

In [5]:
## Support Vector Machines ##

In [7]:
# Function for One_Vs_Rest Svm##

In [8]:
def one_vs_rest_svm(train_x, train_y, test_x):
    """
    Trains a linear SVM for binary classifciation

    Args:
        train_x - (n, d) NumPy array (n datapoints each with d features)
        train_y - (n, ) NumPy array containing the labels (0 or 1) for each training data point
        test_x - (m, d) NumPy array (m datapoints each with d features)
    Returns:
        pred_test_y - (m,) NumPy array containing the labels (0 or 1) for each test data point
    """
    svm_model=LinearSVC(random_state=0,C=0.1)
    svm_model.fit(train_x,train_y)
    prediction=svm_model.predict(test_x)
    return prediction
    raise NotImplementedError

In [9]:
## Multiclass SVM ##

In [10]:
def multi_class_svm(train_x, train_y, test_x):
    """
    Trains a linear SVM for multiclass classifciation using a one-vs-rest strategy

    Args:
        train_x - (n, d) NumPy array (n datapoints each with d features)
        train_y - (n, ) NumPy array containing the labels (int) for each training data point
        test_x - (m, d) NumPy array (m datapoints each with d features)
    Returns:
        pred_test_y - (m,) NumPy array containing the labels (int) for each test data point
    """
    svm_model=LinearSVC(random_state=0,C=0.1,multi_class="ovr")
    svm_model.fit(train_x,train_y)
    prediction=svm_model.predict(test_x)
    return prediction
    raise NotImplementedError


In [11]:
## Multinomial Regression (SOFTMAX REGRESSION) ##

In [12]:
## Function for Computing Probabilities ##

In [13]:
def compute_probabilities(X, theta, temp_parameter):
    """
    Computes, for each datapoint X[i], the probability that X[i] is labeled as j
    for j = 0, 1, ..., k-1

    Args:
        X - (n, d) NumPy array (n datapoints each with d features)
        theta - (k, d) NumPy array, where row j represents the parameters of our model for label j
        temp_parameter - the temperature parameter of softmax function (scalar)
    Returns:
        H - (k, n) NumPy array, where each entry H[j][i] is the probability that X[i] is labeled as j
    """
    #YOUR CODE HERE
    e_exp = np.matmul(theta, np.transpose(X) / temp_parameter)
    c = np.amax(e_exp,axis=0)
    h_num = np.exp(e_exp - c)
    h_den = np.sum(np.exp(e_exp -c),axis=0)
    H = np.divide(h_num, h_den)
    return H
    raise NotImplementedError

In [14]:
## Function for Cost Function ##

In [15]:
def compute_cost_function(X, Y, theta, lambda_factor, temp_parameter):
    """
    Computes the total cost over every datapoint.

    Args:
        X - (n, d) NumPy array (n datapoints each with d features)
        Y - (n, ) NumPy array containing the labels (a number from 0-9) for each
            data point
        theta - (k, d) NumPy array, where row j represents the parameters of our
                model for label j
        lambda_factor - the regularization constant (scalar)
        temp_parameter - the temperature parameter of softmax function (scalar)

    Returns
        c - the cost value (scalar)
    """
    #YOUR CODE HERE
    k = theta.shape[0]
    
    # Get number of examples
    n = X.shape[0]
    
    # avg error term
    
    # Clip prob matrix to avoid NaN instances
    clip_prob_matrix = np.clip(compute_probabilities(X, theta, temp_parameter), 1e-15, 1-1e-15)
    
    # Take the log of the matrix of probabilities
    log_clip_matrix = np.log(clip_prob_matrix)
    
    # Create a sparse matrix of [[y(i) == j]]
    M = sparse.coo_matrix(([1]*n, (Y, range(n))), shape = (k,n)).toarray()
    
    # Only add terms of log(matrix of prob) where M == 1
    error_term = (-1/n)*np.sum(log_clip_matrix[M == 1])    
                
    # Regularization term
    reg_term = (lambda_factor/2)*np.linalg.norm(theta)**2
    
    return error_term + reg_term
    raise NotImplementedError

In [16]:
## Function For Gradient Descent ##

In [17]:
def run_gradient_descent_iteration(X, Y, theta, alpha, lambda_factor, temp_parameter):
    """
    Runs one step of batch gradient descent

    Args:
        X - (n, d) NumPy array (n datapoints each with d features)
        Y - (n, ) NumPy array containing the labels (a number from 0-9) for each
            data point
        theta - (k, d) NumPy array, where row j represents the parameters of our
                model for label j
        alpha - the learning rate (scalar)
        lambda_factor - the regularization constant (scalar)
        temp_parameter - the temperature parameter of softmax function (scalar)

    Returns:
        theta - (k, d) NumPy array that is the final value of parameters theta
    """
    #YOUR CODE HERE
    k = theta.shape[0]
    
    # Get number of examples
    n = X.shape[0]
    
    # Create spare matrix of [[y(i) == j]]
    M = sparse.coo_matrix(([1]*n, (Y, range(n))), shape=(k,n)).toarray()
    
    # Matrix of Probabilities
    P = compute_probabilities(X, theta, temp_parameter)
    
    # Gradient matrix of theta

    grad_theta = (-1/(temp_parameter*n))*(np.matmul((M-P),X)) + lambda_factor*theta
    
    # Gradient descent update of theta matrix
    theta = theta - alpha*grad_theta
    
    return theta
    raise NotImplementedError

In [18]:
## Changing Labels Function ##

In [19]:
def update_y(train_y, test_y):
    """
    Changes the old digit labels for the training and test set for the new (mod 3)
    labels.

    Args:
        train_y - (n, ) NumPy array containing the labels (a number between 0-9)
                 for each datapoint in the training set
        test_y - (n, ) NumPy array containing the labels (a number between 0-9)
                for each datapoint in the test set

    Returns:
        train_y_mod3 - (n, ) NumPy array containing the new labels (a number between 0-2)
                     for each datapoint in the training set
        test_y_mod3 - (n, ) NumPy array containing the new labels (a number between 0-2)
                    for each datapoint in the test set
    """
    #YOUR CODE HERE
    train_y_mod3=np.remainder(train_y,3)
    test_y_mod3=np.remainder(test_y,3)
    return train_y_mod3,test_y_mod3
    raise NotImplementedError


In [20]:
def compute_test_error_mod3(X, Y, theta, temp_parameter):
    """
    Returns the error of these new labels when the classifier predicts the digit. (mod 3)

    Args:
        X - (n, d - 1) NumPy array (n datapoints each with d - 1 features)
        Y - (n, ) NumPy array containing the labels (a number from 0-2) for each
            data point
        theta - (k, d) NumPy array, where row j represents the parameters of our
                model for label j
        temp_parameter - the temperature parameter of softmax function (scalar)

    Returns:
        test_error - the error rate of the classifier (scalar)
    """
    #YOUR CODE HERE
    y_pred = get_classification(X, theta, temp_parameter)
    
    return 1 - (np.mod(y_pred, 3) == Y).mean()
    raise NotImplementedError


In [21]:
## Dimensionality Reduction Using Principal Component Analysis ##

In [22]:
def project_onto_PC(X, pcs, n_components, feature_means):
    """
    Given principal component vectors pcs = principal_components(X)
    this function returns a new data array in which each sample in X
    has been projected onto the first n_components principcal components.
    """
    # TODO: first center data using the feature_means
    # TODO: Return the projection of the centered dataset
    #       on the first n_components principal components.
    #       This should be an array with dimensions: n x n_components.
    # Hint: these principal components = first n_components columns
    #       of the eigenvectors returned by principal_components().
    #       Note that each eigenvector is already be a unit-vector,
    #       so the projection may be done using matrix multiplication.
    import numpy as np
    X_centered = X-feature_means
    
    V_n = pcs[:,0:n_components]
    
    projected_data = np.matmul(X_centered,V_n)
    
    return projected_data
    raise NotImplementedError

In [23]:
## Kernels Method ##

In [24]:
## Function for Polynomial Kernel ##

In [25]:
def polynomial_kernel(X, Y, c, p):
    """
        Compute the polynomial kernel between two matrices X and Y::
            K(x, y) = (<x, y> + c)^p
        for each pair of rows x in X and y in Y.

        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            c - a coefficient to trade off high-order and low-order terms (scalar)
            p - the degree of the polynomial kernel

        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    # YOUR CODE HERE
    kernel_matrix = (X.dot(Y.T) + c)**p
    
    return kernel_matrix
    raise NotImplementedError

In [26]:
## Gaussian RBF Kernel ## 

In [27]:
def rbf_kernel(X, Y, gamma):
    """
        Compute the Gaussian RBF kernel between two matrices X and Y::
            K(x, y) = exp(-gamma ||x-y||^2)
        for each pair of rows x in X and y in Y.
        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            gamma - the gamma parameter of gaussian function (scalar)
        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    n, d = X.shape
    m = Y.shape[0]

    # Naive version with for loops   
#    kernel_matrix = np.zeros((n,m))
#    
##### Single for loop ####
#    for i in range(n):
#        d = X[i,:] - Y  # Using numpy broadcasting to get differences
#        b = np.sum(d**2, axis=1)
#        kernel_matrix[i,:] = np.exp(-gamma*b)
#### End: single-loop version ####

#### Vectorized version (runs slower than the single loop version) ####
#    kernel_matrix = np.linalg.norm((X[:,None] - Y), ord=2, axis=2)**2
#    kernel_matrix = (np.sum(X**2, axis=1)[:,None] + np.sum(Y**2, axis=1)) - 2*(X @ Y.T)
    kernel_matrix = X**2 @ np.ones((d,m)) + np.ones((n,d)) @ Y.T**2 - 2*(X @ Y.T)
    kernel_matrix = np.exp(-gamma*kernel_matrix)
#### End: vectorized version ####
    
    return kernel_matrix